In [1]:
# adds parent dir to python path
import sys
sys.path.insert(0, '..')

#system lib
import os
import json

# 3rd party lib
import numpy as np
#import matplotlib
#import matplotlib.pyplot as plt
#%matplotlib notebook
#matplotlib.rcParams.update({'font.size': 22})
#from bqplot import pyplot as plt

import matplotlib.pyplot as plt
%matplotlib auto

# astro lib
from astropy.io import fits
import sep

# my lib
import ImageTools as it
import DataTools as dt
from gphelper import GPHelper

from importlib import reload

import warnings
warnings.filterwarnings('ignore')


from multiprocessing import Pool

Using matplotlib backend: Qt5Agg


In [332]:
def make_nonnegative(img):
    epsilon = np.abs(img.min()) + 1e-3
    if img.min() <= 0:
        img += epsilon
    return img

def get_spheroids(num=10):
    with open('../spheroids', 'r') as f:
        spheroids= np.array(f.readlines())
    
    data_dir = os.path.join(os.getenv('HOME'), 'Documents/astro_data/orig_images')
    fmask = 'GDS_{}_{}.fits'
    f_string = os.path.join(data_dir, fmask)
    
    sources = []
    for s in spheroids:
        s = s.strip()
        img = fits.getdata(f_string.format(s, 'h'))
        segmap = fits.getdata(f_string.format(s, 'segmap'))
        img_id = int(s.split('_')[1])
        sources.append((img.copy(), segmap.copy(), img_id))
        del img
        del segmap
        
    return sources
        
def process_img(args):
    def denoise(img, segmap, img_id):
        noise_bank = img[segmap==0].flatten()
        other_source = np.logical_and(segmap!=0, segmap!=img_id)
        np.place(img, other_source, noise_bank)

        sep_arr = img.byteswap().newbyteorder('N')
        bkg = sep.Background(sep_arr, mask=segmap==img_id, bw=10, bh=10)
        img = img-bkg

        return img
    
    def get_rs_and_fs(img, src_map):
        cx, cy = it.img_center(img, src_map)
        xs, ys = np.meshgrid(np.arange(img.shape[0]), np.arange(img.shape[1]).T)
        rs = np.sqrt(np.square(xs-cx)+np.square(ys-cy))

        rs = rs.flatten()
        fs = img.flatten()
        sorted_rs = np.argsort(rs)

        rs = rs[sorted_rs]
        fs = fs[sorted_rs]

        return rs, fs
    
    img, segmap, img_id = args
    src_map = segmap==img_id
    img = denoise(img, segmap, img_id)
    
    rs, fs = get_rs_and_fs(img, src_map)
    
    dr = 0.05*(rs.max()-rs.min())
    fs = dt.loessc_p(rs, fs, dr, pnum=2)
    fs[fs<0]=0
    
    num_interp = 100
    _rs = np.linspace(rs.min(), rs.max(), num_interp)
    fs = np.interp(_rs, rs, fs)
    rs = _rs
    
    L = np.zeros(num_interp)
    L[0] = np.pi*rs[0]**2  * fs[0]
    for i in range(1,num_interp,1):
        L[i] = L[i-1] + np.pi*(rs[i]**2 - rs[i-1]**2)*((fs[i]+fs[i-1])/2)

    A = np.pi * rs**2
    etas = fs*A/L
    r_lim = rs<25.0
    #etas = etas[r_lim]
    mono_dec = np.argmax(np.diff(etas)>=0)
    mono_dec = np.arange(len(etas))<mono_dec
    r_lim = np.logical_and(r_lim, mono_dec)
    
    if np.all(r_lim==False):
        #plt.plot(rs, fs)
        plt.figure()
        plt.plot(rs[:-1],np.diff(etas))
        plt.figure()
        plt.plot(rs, etas)
        #plt.figure()
        #plt.plot(np.flipud(etas), np.flipud(rs))
        #plt.figure()
        #plt.title(img_id)
        #plt.imshow(img)
    
    Rp = np.interp(0.2, np.flipud(etas[r_lim]), np.flipud(rs[r_lim]))
    Fp_idx = np.square(rs - 2*Rp).argmin()

    Fp = np.cumsum(L[:Fp_idx]/L[:Fp_idx].sum())
    Ip = np.interp(Rp, rs, fs)

    return (rs, fs, Rp, Ip, etas, img_id)

In [399]:
h_vals = []

for s in processed:
    v = {
        'rs' : dt._nmpy_encode(s[0]),
        'fs' : dt._nmpy_encode(s[1]),
        'rp' : s[2],
        'ip' : s[3],
        'etas' : dt._nmpy_encode(s[4]),
        'id' : s[5]
    }
    h_vals.append(v)
    
with open('h_vals.json', 'w') as f:
    json.dump(h_vals, f)

In [398]:
for k in h_vals[0].keys():
    print(k, type(h_vals[0][k]))
    
#print(h_vals[0])

rs <class 'str'>
fs <class 'str'>
rp <class 'float'>
ip <class 'float'>
etas <class 'str'>
id <class 'int'>


In [3]:
spheroids = get_spheroids(num=-1)

In [367]:
rms = []
for img, segmap, img_id in spheroids[:]:
    noise = img[segmap==0].flatten()
    rms.append(np.sqrt(np.mean(noise**2)))
rms = np.array(rms)

In [377]:
plt.figure()
plt.hist(rms[rms<0.01], bins=50)
plt.title('Histogram of RMS Measured From Noise')
plt.xlabel('RMS(Image Noise)')

In [334]:
%%time

count = 0
processed = []
for s in spheroids[:]:
    print(count/len(spheroids), end='\r')
    count += 1
    if s[2]==2845:
        continue
    processed.append(process_img(s))

CPU times: user 2min 16s, sys: 42.8 s, total: 2min 59s
Wall time: 21min 43s


In [338]:
post_processed = []
for p in processed:
    if p[2]<5:
        continue
        
    post_processed.append(p)
    
processed = post_processed

In [339]:
plt.title('1/$\eta(R)$ and $R/R_p$ - Data')
plt.ylabel('1/$\eta(R)$')
plt.xlabel('$R/R_p$')
for (rs, fs, Rp, Ip, etas, _) in processed:
    plt.plot(rs/Rp, etas, color='b', alpha=0.2)

In [341]:
interped_marks = np.concatenate([np.linspace(0, 1, 50, endpoint=False), np.linspace(1, 20, 950)])

stat_vals = []

plt.figure()
plt.title('Normalized Surface Brightness')
plt.xlabel('$R/R_p$')
plt.ylabel('$I/I_p$ (Log)')

for (rs, fs, Rp, Ip, etas, _) in processed:
    color = 'b'
    if np.sum(fs<=0) > 0:
        fs = fs + np.abs(fs.min()) + 1e-3
        Ip = np.interp(Rp, rs, fs)
    
    r = rs/Rp
    f = fs/Ip
    plt.plot(r, np.log10(f), color='k', alpha=.05)
    stat_vals.append(np.interp(interped_marks, r, f, left=np.nan, right=np.nan))
    

stat_vals = np.log10(np.dstack(stat_vals)[0,...])
f_mean = np.nanmedian(stat_vals, axis=1)
f_16 = np.nanpercentile(stat_vals, 16, axis=1)
f_84 = np.nanpercentile(stat_vals, 84, axis=1)
    

plt.plot(interped_marks, f_mean, color='r', label='$median$', zorder=100)
plt.fill_between(interped_marks, f_16, f_84, color='r', alpha=0.45, label="$16^{th}-84^{th}$", zorder=100)
plt.legend()
plt.show()

In [342]:
interped_marks = np.concatenate([np.linspace(0, 1, 50, endpoint=False), np.linspace(1, 20, 950)])
#stat_vals = np.dstack(stat_vals)[0,...]
f_mean = np.nanmedian(stat_vals, axis=1)
f_16 = np.nanpercentile(stat_vals, 16, axis=1)
f_84 = np.nanpercentile(stat_vals, 84, axis=1)

plt.figure()
plt.plot(interped_marks, f_mean, color='r', label='$median$', zorder=100)
plt.fill_between(interped_marks, f_16, f_84, color='r', alpha=0.45, label="$16^{th}-84^{th}$", zorder=100)
plt.legend()
plt.show()

In [343]:
stat_vals.shape

(1000, 496)

In [344]:
valid_points = ~np.isnan(stat_vals).all(axis=1)

X = interped_marks[valid_points]
Y = f_mean[valid_points]
a = f_84[valid_points] - f_mean[valid_points]

vals = {'x':dt._nmpy_encode(X),
        'y':dt._nmpy_encode(Y),
        'a':dt._nmpy_encode(a)}
with open('vals_for_gp.json', 'w') as f:
    json.dump(vals,f)

In [21]:
with open('vals_for_gp.json', 'r') as f:
    vals = json.load(f)
    
X = dt._nmpy_decode(vals['x'])
Y = dt._nmpy_decode(vals['y'])
a = dt._nmpy_decode(vals['a'])

print(X.shape, Y.shape, a.shape)

(464,) (464,) (464,)


In [345]:
# truncate lines from 5re and on
upto = 5
_X = X[X<=upto]
_Y = Y[X<=upto]
_a = a[X<=upto]
tmp = _Y-_a


plt.figure()

print(np.isnan(_X).sum(), np.isnan(_Y).sum(), np.isnan(_a).sum())
split_idx = 2#np.argmin(np.diff(_a))
split_val = _X[split_idx]

# fit and pad line to smooth
plt.plot(_X[split_idx:],_Y[split_idx:], label='Measured Values')
plt.fill_between(_X[split_idx:], tmp[split_idx:], _Y[split_idx:]+_a[split_idx:], alpha=0.2)

_X = _X[split_idx:]
_Y = _Y[split_idx:]
_a = _a[split_idx:]

_X, _Y = dt.pad_line(_X, _Y, 10, 10, append=False)
_, _a = dt.pad_line(_X, _a, 10, 10, append=False)

plt.plot(_X[_X<split_val], _Y[_X<split_val], '--', label='Linear Fit Values')
plt.fill_between(_X[_X<split_val], _Y[_X<split_val]-_a[_X<split_val], _Y[_X<split_val]+_a[_X<split_val], alpha=0.2)

plt.xlabel('$R/R_e$')
plt.ylabel('$I/I_e (Log)$')
plt.title('Measured Surface Brightness W/Linear Fit Projection')
plt.legend()

0 0 0


In [346]:
gp = GPHelper()
gp.fit(_X[:,np.newaxis], _Y, _a, optimize='both')
_x, std = gp.predict(_X[:, np.newaxis], return_std=True)

In [348]:
plt.figure()
plt.plot(_X,_Y, label='GP Fit')
plt.fill_between(_X, _Y-std, _Y+std, alpha=0.2)
plt.legend()

In [296]:
if gp is None:
    gp = GPHelper(restore_file='gp.json')
else:
    gp.save_params()

In [388]:
gp.save_params()

In [378]:
test_vals = np.concatenate([np.linspace(0.01, 1, 20, endpoint=False), np.linspace(1, upto, upto*20)])

In [379]:
#samples = gp.sample(test_vals[:,np.newaxis], num_samples=500)
samples = []
for i in range(500):
    print(i/500)
    samples.append(gp.sample(test_vals[:,np.newaxis]))

0.0
0.002ng Samples...  
0.004ng Samples..   
0.006ng Samples...  
0.008ng Samples...  
0.01ing Samples...  
0.012ng Samples     
0.014ng Samples...  
0.016ng Samples...  
0.018ng Samples..   
0.02ing Samples...  
0.022ng Samples...  
0.024ng Samples..   
0.026ng Samples     
0.028ng Samples     
0.03ing Samples...  
0.032ng Samples.    
0.034ng Samples     
0.036ng Samples..   
0.038ng Samples.    
0.04ing Samples     
0.042ng Samples.    
0.044ng Samples...  
0.046ng Samples     
0.048ng Samples     
0.05ing Samples     
0.052ng Samples.    
0.054ng Samples     
0.056ng Samples..   
0.058ng Samples..   
0.06ing Samples..   
0.062ng Samples.    
0.064ng Samples..   
0.066ng Samples.    
0.068ng Samples...  
0.07ing Samples...  
0.072ng Samples..   
0.074ng Samples     
0.076ng Samples...  
0.078ng Samples.    
0.08ing Samples     
0.082ng Samples..   
0.084ng Samples...  
0.086ng Samples..   
0.088ng Samples.    
0.09ing Samples     
0.092ng Samples.    
0.094ng Samples     
0.096ng S

In [380]:
samples = np.stack(samples)

In [381]:
samples.shape

(500, 120, 1)

In [382]:
np.percentile(samples, 16, axis=0).shape

(120, 1)

In [383]:
plt.figure()
plt.title('GP 500 Samples')
plt.xlabel('$R/R_p$')
plt.ylabel('$I/I_p$ (Log)')

for i in range(samples.shape[0]):
    plt.plot(test_vals, samples[i,:,0], color='b', alpha=0.2)

In [384]:
x = test_vals
gp_res = []
etas_rrp = []
rp_comp = []
plt.figure()
plt.title("R vs 1/$\eta(R)$")
plt.xlabel("1/$\eta(R)$")
plt.ylabel("R")
for (rs, _, Rp, Ip, etas, _), fs in zip(processed, list(samples[:,:,0])):
    fs = 10**fs * Ip
    rs = x * Rp

    L = np.zeros_like(fs)
    L[0] = np.pi*rs[0]**2  * fs[0]
    for i in range(1,len(fs),1):
        L[i] = L[i-1] + np.pi*(rs[i]**2 - rs[i-1]**2)*((fs[i]+fs[i-1])/2)

    A = np.pi * rs**2
    etas = fs*A/L
    r_lim = rs<25.0
    mono_dec = np.argmax(np.diff(etas)>=0)
    mono_dec = np.arange(len(etas))<mono_dec
    r_lim = np.logical_and(r_lim, mono_dec)
    
    R_p = np.interp(0.2, np.flipud(etas[r_lim]), np.flipud(rs[r_lim]))
    plt.plot(etas[r_lim], rs[r_lim])

    gp_res.append((Rp, R_p))
    etas_rrp.append((rs/R_p, etas))
    rp_comp.append((Rp, R_p))

In [385]:
plt.figure()
plt.title('Data $R_p$ vs Model $R_p$')
plt.xlabel('Data $R_p$')
plt.ylabel('Model $R_p$ / Data $R_p$')
plt.ylim((0, 2))

data_rp, model_rp = zip(*rp_comp)
plt.scatter(data_rp, np.array(model_rp)/np.array(data_rp))
plt.hlines(1, 0, 20)
#plt.plot()
#mx = np.ceil(max(max(r), max(gr)))
#mn = np.floor(min(min(r), min(gr)))
#plt.plot(np.arange(5, 21), np.arange(5, 21), color='c')
plt.legend()

In [359]:
plt.figure()
plt.title('1/$\eta(R)$ and $R/R_p$ - Model')
plt.ylabel('1/$\eta(R)$')
plt.xlabel('$R/R_p$')
for r, f in etas_rrp:
    plt.plot(r, f, color='b', alpha=0.2)

In [386]:
plt.figure()
r, gr = zip(*gp_res)
plt.title('Input $R_p$ vs Measured $R_p$ - GP Generated SBP')
plt.xlabel('Input $R_p$')
plt.ylabel('Measured $R_p$')
plt.scatter(r, gr, color='b')
mx = np.ceil(max(max(r), max(gr)))
mn = np.floor(min(min(r), min(gr)))
plt.plot(np.arange(mn, mx), np.arange(mn, mx), color='c')

In [361]:
plt.figure()
r, gr = zip(*gp_res)
plt.hist(r, bins=40, alpha=0.5, label='Data')
#plt.hist(new_res, bins=40, alpha=0.5, label='Line Measured')
#plt.figure()
plt.hist(gr, bins=40, alpha=0.5, label='GP')
plt.xlabel("Petrosian Radius")
plt.legend()

In [75]:
xs, ys = np.meshgrid(np.arange(84), np.arange(84))
cxy = 42
rs = np.sqrt((xs-cxy)**2 + (ys-cxy)**2)

s = samples[5,:]
_, _, re, ie = processed[35]
rs = rs/re
fs = s*ie


fs = np.interp(rs.flatten(), test_vals, fs).reshape([84,84])
plt.imshow(fs, cmap='gray')

In [76]:
img, segmap, img_id = spheroids[1]

noise = img[segmap==0]

frame = np.zeros([84,84])
np.place(frame, np.ones_like(frame), noise)
np.random.shuffle(frame)
plt.imshow(frame, cmap='gray')

In [81]:
plt.imshow(fs+(frame*0.06), cmap='gray')

In [82]:
fits.PrimaryHDU(frame).writeto('noise_bank.fits')
fits.PrimaryHDU(fs).writeto('gp_img.fits')
fits.PrimaryHDU(fs+(frame*0.06)).writeto('gp_with_noise.fits')